In [61]:
import pandas as pd
import numpy as np

In [62]:
# walking_df = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\6월14일_IMU_walking_processed.csv')
running_df = pd.read_csv(r'C:\Users\User\Desktop\6월11일_마지막\6월14일_IMU_jumping_processed.csv')

# walking_df['Activity'] = 'walking'
running_df['Activity'] = 'jumping'

# df = pd.concat([running_df, walking_df], axis = 0)
df = running_df.copy()
df.head()

,angular_velocity_x,angular_velocity_y,angular_velocity_z,linear_acceleration_x,linear_acceleration_y,linear_acceleration_z,Activity
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping


In [63]:
cols = ['Angular-X', 'Angular-Y', 'Angular-Z', 'Linear-X', 'Linear-Y', 'Linear-Z', 'Activity']
df.columns = cols

df.head()


,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping


In [64]:
# 이상치 제거 -> IQR (파생변수 생성 전, 이상치 제거를 하는게 맞음)
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

columns = ['Angular-X', 'Angular-Y', 'Angular-Z', 'Linear-X', 'Linear-Y', 'Linear-Z']

df = remove_outliers_iqr(df, columns)

df.head().reset_index(drop=True)

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping


In [65]:
# 각 축의 크기
def calculate_magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

# Angular 및 Linear 속성의 크기 계산하여 새로운 컬럼 추가
df['Angular-Magnitude'] = df.apply(lambda row: calculate_magnitude(row['Angular-X'], row['Angular-Y'], row['Angular-Z']), axis=1)
df['Linear-Magnitude'] = df.apply(lambda row: calculate_magnitude(row['Linear-X'], row['Linear-Y'], row['Linear-Z']), axis=1)

df.head()

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity,Angular-Magnitude,Linear-Magnitude
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping,0.796755,9.982893
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping,0.706666,10.183057
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping,0.609576,10.434167
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping,0.511694,10.506099
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping,0.444722,10.405661


In [66]:
# 각 축의 변동성 계산
df['Angular-Var'] = df[['Angular-X', 'Angular-Y', 'Angular-Z']].var(axis=1)
df['Linear-Var'] = df[['Linear-X', 'Linear-Y', 'Linear-Z']].var(axis=1)

df.head()

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping,0.796755,9.982893,0.223188,28.257022
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping,0.706666,10.183057,0.176381,28.132693
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping,0.609576,10.434167,0.134645,29.392378
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping,0.511694,10.506099,0.090296,31.397579
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping,0.444722,10.405661,0.063996,31.745471


In [67]:
# 각 축의 frequency domain
def calculate_fft(data):
    return np.abs(np.fft.rfft(data))

df['Angular-FFT'] = df.apply(lambda row: calculate_fft([row['Angular-X'], row['Angular-Y'], row['Angular-Z']]), axis=1)
df['Linear-FFT'] = df.apply(lambda row: calculate_fft([row['Linear-X'], row['Linear-Y'], row['Linear-Z']]), axis=1)

df.head()

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Angular-FFT,Linear-FFT
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping,0.796755,9.982893,0.223188,28.257022,"[0.7518821060657501, 0.81826958377228]","[11.376833491874514, 9.207120370780897]"
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping,0.706666,10.183057,0.176381,28.132693,"[0.6632086634635925, 0.7274221283730143]","[11.928443957390407, 9.186842727195517]"
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping,0.609576,10.434167,0.134645,29.392378,"[0.5539652556180954, 0.635559082225951]","[12.258110657280454, 9.390268051712356]"
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping,0.511694,10.506099,0.090296,31.397579,"[0.4936763197183609, 0.5204692634096431]","[11.94775601306804, 9.705294211919734]"
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping,0.444722,10.405661,0.063996,31.745471,"[0.45755207538604736, 0.4381654593473577]","[11.591398695958027, 9.758914485238263]"


In [68]:
# 가속도와 각속도의 합
df['Total-Magnitude'] = df['Angular-Magnitude'] + df['Linear-Magnitude']

df.head()

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Angular-FFT,Linear-FFT,Total-Magnitude
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping,0.796755,9.982893,0.223188,28.257022,"[0.7518821060657501, 0.81826958377228]","[11.376833491874514, 9.207120370780897]",10.779649
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping,0.706666,10.183057,0.176381,28.132693,"[0.6632086634635925, 0.7274221283730143]","[11.928443957390407, 9.186842727195517]",10.889723
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping,0.609576,10.434167,0.134645,29.392378,"[0.5539652556180954, 0.635559082225951]","[12.258110657280454, 9.390268051712356]",11.043743
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping,0.511694,10.506099,0.090296,31.397579,"[0.4936763197183609, 0.5204692634096431]","[11.94775601306804, 9.705294211919734]",11.017793
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping,0.444722,10.405661,0.063996,31.745471,"[0.45755207538604736, 0.4381654593473577]","[11.591398695958027, 9.758914485238263]",10.850383


In [69]:
# 파생변수 다 만듬 -> 결측치 확인
df.isnull().sum()

Angular-X            0
Angular-Y            0
Angular-Z            0
Linear-X             0
Linear-Y             0
Linear-Z             0
Activity             0
Angular-Magnitude    0
Linear-Magnitude     0
Angular-Var          0
Linear-Var           0
Angular-FFT          0
Linear-FFT           0
Total-Magnitude      0
dtype: int64

In [70]:
# Strength와 Phase로 나눠야함 ([Strength, Phase]로 나눠지기 때문)
df['Angular-FFT-Strength'] = df['Angular-FFT'].apply(lambda x: x[0])
df['Angular-FFT-Phase'] = df['Angular-FFT'].apply(lambda x: x[1])
df['Linear-FFT-Strength'] = df['Linear-FFT'].apply(lambda x: x[0])
df['Linear-FFT-Phase'] = df['Linear-FFT'].apply(lambda x: x[1])

# 기존의 FFT 변수를 제거
df.drop(columns=['Angular-FFT', 'Linear-FFT'], inplace=True)

df.head()

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Total-Magnitude,Angular-FFT-Strength,Angular-FFT-Phase,Linear-FFT-Strength,Linear-FFT-Phase
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping,0.796755,9.982893,0.223188,28.257022,10.779649,0.751882,0.818270,11.376833,9.207120
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping,0.706666,10.183057,0.176381,28.132693,10.889723,0.663209,0.727422,11.928444,9.186843
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping,0.609576,10.434167,0.134645,29.392378,11.043743,0.553965,0.635559,12.258111,9.390268
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping,0.511694,10.506099,0.090296,31.397579,11.017793,0.493676,0.520469,11.947756,9.705294
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping,0.444722,10.405661,0.063996,31.745471,10.850383,0.457552,0.438165,11.591399,9.758914


In [71]:
# 인덱스 다시 정렬
df = df.reset_index(drop = True)
df.head()

,Angular-X,Angular-Y,Angular-Z,Linear-X,Linear-Y,Linear-Z,Activity,Angular-Magnitude,Linear-Magnitude,Angular-Var,Linear-Var,Total-Magnitude,Angular-FFT-Strength,Angular-FFT-Phase,Linear-FFT-Strength,Linear-FFT-Phase
0,-0.204257,0.217304,0.738835,9.237516,3.523036,-1.383718,jumping,0.796755,9.982893,0.223188,28.257022,10.779649,0.751882,0.818270,11.376833,9.207120
1,-0.205510,0.234602,0.634117,9.345364,3.843272,-1.260192,jumping,0.706666,10.183057,0.176381,28.132693,10.889723,0.663209,0.727422,11.928444,9.186843
2,-0.214675,0.261655,0.506985,9.513398,4.074243,-1.329530,jumping,0.609576,10.434167,0.134645,29.392378,11.043743,0.553965,0.635559,12.258111,9.390268
3,-0.179620,0.298543,0.374753,9.601877,3.950573,-1.604694,jumping,0.511694,10.506099,0.090296,31.397579,11.017793,0.493676,0.520469,11.947756,9.705294
4,-0.136306,0.334770,0.259089,9.506187,3.847615,-1.762402,jumping,0.444722,10.405661,0.063996,31.745471,10.850383,0.457552,0.438165,11.591399,9.758914


In [72]:
df.to_csv('6월14일_IMU_jumping_cleaned.csv')